In [29]:
import numpy as np
import pandas as pd
from sklearn.externals import joblib
import gc

In [30]:
with np.load('data/interim/kmers/kmer_matrix.npz') as data:
    kmers = data['kmers']
    kmer_order = data['kmer_order']
    genome_order = data['genome_order']

In [31]:
micsdf = joblib.load('data/interim/mic_class_dataframe2.pkl')
class_orders = joblib.load('data/interim/mic_class_order_dict2.pkl')

In [32]:
# TIO
tio_labels = class_orders['TIO']
tio_label_index = { k: v for v, k in enumerate(tio_labels) }
y_tio = np.array([ tio_label_index[m] if not pd.isna(m) else m for m in micsdf.loc[genome_order, 'TIO'] ])
labels, counts = np.unique(y_tio, return_counts=True)
ok = labels[counts >= 5]

mask = np.in1d(y_tio, ok) # Since Nan is not a label, this also filters invalid MICs
y_tio = y_tio[mask]
X_tio = kmers[mask,:]
tio_samples = genome_order[mask]
del kmers
del genome_order
del micsdf
del class_orders
gc.collect()
# %xdel kmers
# %xdel genome_order
# %xdel micsdf
# %xdel class_orders

6715

In [34]:
# Test/train split
from sklearn.model_selection import train_test_split
indices = np.arange(len(tio_samples))
X_train, X_test, y_train, y_test, idx1, idx2 = train_test_split(X_tio, y_tio, indices, test_size=0.2, random_state=36, stratify=y_tio)
del X_tio
del y_tio
gc.collect()

290

In [6]:
print(np.unique(y_train,return_counts=True))
print(np.unique(y_test,return_counts=True))

(array([1., 2., 3., 4., 5., 6.]), array([ 404, 1046,   66,    6,   78,  205]))
(array([1., 2., 3., 4., 5., 6.]), array([101, 262,  16,   1,  20,  52]))


In [7]:
# Feature selection
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif

fsel = SelectKBest(f_classif, k=10000)
X_train_fs = fsel.fit_transform(X_train, y_train)
X_test_fs = fsel.transform(X_test)



/home/matt/anaconda3/envs/phenores/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [ 149082  195348  257056 ... 2008529 2008530 2008531] are constant.
  UserWarning)
/home/matt/anaconda3/envs/phenores/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw
/home/matt/anaconda3/envs/phenores/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [8]:
import tensorflow
from tensorflow import set_random_seed

from keras.layers.core import Dense, Dropout, Activation

from keras.models import Sequential#, load_model
from keras.utils import np_utils, to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


Using TensorFlow backend.


In [15]:
patience = 16
early_stop = EarlyStopping(monitor='loss', patience=patience, verbose=0, min_delta=0.005, mode='auto')
model_save = ModelCheckpoint("best_model.hdf5",monitor='loss', verbose = 0, save_best_only =True, save_weights_only = False, mode ='auto', period =1)
reduce_LR = ReduceLROnPlateau(monitor='loss', factor= 0.1, patience=(patience/2), verbose = 0, min_delta=0.005,mode = 'auto', cooldown=0, min_lr=0)

num_classes=6
n1 = X_test_fs.shape[1]
n2 = int((n1+num_classes)/2)
model = Sequential()
model.add(Dense(n1,activation='relu',input_dim=(n1)))
model.add(Dropout(0.5))
model.add(Dense(n2, activation='relu', kernel_initializer='uniform'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='relu', kernel_initializer='uniform', ))
model.add(Dense(1, kernel_initializer='normal'))

model.compile(loss='mae', metrics=['accuracy'], optimizer='adam')

In [19]:
history = model.fit(X_train_fs, y_train, epochs=100, batch_size=200, verbose=1, shuffle=True, validation_split=0.33, callbacks=[early_stop, reduce_LR])

Epoch 1/100


 200/1805 [==>...........................] - ETA: 18s - loss: 0.9308 - acc: 0.4250

 400/1805 [=====>........................] - ETA: 16s - loss: 1.1585 - acc: 0.3200

 600/1805 [========>.....................] - ETA: 13s - loss: 1.1449 - acc: 0.3067

 800/1805 [============>.................] - ETA: 11s - loss: 1.0759 - acc: 0.3438

...

1200/1805 [==================>...........] - ETA: 6s - loss: 0.3371 - acc: 0.7733

1400/1805 [======================>.......] - ETA: 4s - loss: 0.3384 - acc: 0.7671

1600/1805 [=========================>....] - ETA: 2s - loss: 0.3427 - acc: 0.7600

1800/1805 [============================>.] - ETA: 0s - loss: 0.3402 - acc: 0.7628

1805/1805 [==============================] - 22s 12ms/step - loss: 0.3405 - acc: 0.7623


In [20]:
score = model.evaluate(X_test_fs, y_test, verbose=0)
print(score)
y_pred = model.predict(X_test_fs)

[0.5275447769502623, 0.6216814159292036]


In [78]:
clonaldf = pd.read_csv('data/interim/mash_population_groups.csv', index_col=0)
test_samples = tio_samples[idx2]
test_cgs = clonaldf.loc[test_samples]

resdf = pd.DataFrame(data=np.vstack((y_test, y_pred[:,0], test_cgs.values[:,0])).T,
             index=test_samples,
             columns=['true','pred','group'])
resdf['err'] = np.abs(resdf['true'] - resdf['pred'])

In [83]:
resdf.groupby('group').agg(['min','max','mean','median','count'])

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead tr th {
        text-align: left;
    }

    .dataframe thead tr:last-of-type th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr>
      <th></th>
      <th colspan="5" halign="left">true</th>
      <th colspan="5" halign="left">pred</th>
      <th colspan="5" halign="left">err</th>
    </tr>
    <tr>
      <th></th>
      <th>min</th>
      <th>max</th>
      <th>mean</th>
      <th>median</th>
      <th>count</th>
      <th>min</th>
      <th>max</th>
      <th>mean</th>
      <th>median</th>
      <th>count</th>
      <th>min</th>
      <th>max</th>
      <th>mean</th>
      <th>median</th>
      <th>count</th>
    </tr>
    <tr>
      <th>group</th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>1.0</th>
      <td>1.0</td>
      <td>6.0</td>
      <td>2.935780</td>
      <td>2.0</td>
      <td>109</td>
      <td>1.004448</td>
      <td>5.242149</td>
      <td>2.495739</td>
      <td>1.633258</td>
      <td>109</td>
      <td>0.007659</td>
      <td>1.995552</td>
      <td>0.665296</td>
      <td>0.573331</td>
      <td>109</td>
    </tr>
    <tr>
      <th>2.0</th>
      <td>1.0</td>
      <td>3.0</td>
      <td>2.000000</td>
      <td>2.0</td>
      <td>27</td>
      <td>1.564145</td>
      <td>1.832543</td>
      <td>1.695643</td>
      <td>1.671769</td>
      <td>27</td>
      <td>0.167457</td>
      <td>1.376288</td>
      <td>0.398712</td>
      <td>0.332707</td>
      <td>27</td>
    </tr>
    <tr>
      <th>3.0</th>
      <td>1.0</td>
      <td>6.0</td>
      <td>2.421053</td>
      <td>2.0</td>
      <td>38</td>
      <td>1.390204</td>
      <td>5.212219</td>
      <td>2.225546</td>
      <td>1.635343</td>
      <td>38</td>
      <td>0.326017</td>
      <td>1.395659</td>
      <td>0.584788</td>
      <td>0.538485</td>
      <td>38</td>
    </tr>
    <tr>
      <th>4.0</th>
      <td>2.0</td>
      <td>6.0</td>
      <td>2.307692</td>
      <td>2.0</td>
      <td>39</td>
      <td>1.640253</td>
      <td>5.112052</td>
      <td>1.960864</td>
      <td>1.718330</td>
      <td>39</td>
      <td>0.095933</td>
      <td>1.396342</td>
      <td>0.393892</td>
      <td>0.287627</td>
      <td>39</td>
    </tr>
    <tr>
      <th>5.0</th>
      <td>1.0</td>
      <td>6.0</td>
      <td>2.140845</td>
      <td>2.0</td>
      <td>71</td>
      <td>0.873484</td>
      <td>5.365334</td>
      <td>1.783890</td>
      <td>1.654835</td>
      <td>71</td>
      <td>0.025037</td>
      <td>1.268580</td>
      <td>0.422528</td>
      <td>0.342038</td>
      <td>71</td>
    </tr>
    <tr>
      <th>6.0</th>
      <td>1.0</td>
      <td>6.0</td>
      <td>2.258065</td>
      <td>1.5</td>
      <td>62</td>
      <td>0.761844</td>
      <td>6.159883</td>
      <td>1.670287</td>
      <td>0.883462</td>
      <td>62</td>
      <td>0.022224</td>
      <td>2.117713</td>
      <td>0.592935</td>
      <td>0.232997</td>
      <td>62</td>
    </tr>
    <tr>
      <th>7.0</th>
      <td>1.0</td>
      <td>6.0</td>
      <td>2.395833</td>
      <td>2.0</td>
      <td>48</td>
      <td>1.288478</td>
      <td>5.368525</td>
      <td>2.070530</td>
      <td>1.714174</td>
      <td>48</td>
      <td>0.138168</td>
      <td>1.322258</td>
      <td>0.432686</td>
      <td>0.318300</td>
      <td>48</td>
    </tr>
    <tr>
      <th>8.0</th>
      <td>1.0</td>
      <td>6.0</td>
      <td>2.214286</td>
      <td>2.0</td>
      <td>56</td>
      <td>0.991342</td>
      <td>5.021258</td>
      <td>1.835270</td>
      <td>1.650584</td>
      <td>56</td>
      <td>0.008658</td>
      

In [62]:
.shape

(452,)

In [65]:
y_test.shape

(452,)